# Diagnostic 16: Optuna - NO SQLite (Pure In-Memory)

In [ ]:
%run ../00_setup_and_config

In [ ]:
%pip install optuna --quiet

In [ ]:
import sys, os, pandas as pd, numpy as np, pickle
from datetime import datetime
import optuna
from optuna.samplers import TPESampler
import importlib.util
print("="*80)
print("OPTUNA OPTIMIZATION")
print("="*80)

## Load Strategies

In [ ]:
if "all_strategies_pct" in sys.modules: del sys.modules["all_strategies_pct"]
spec = importlib.util.spec_from_file_location("all_strategies_pct", "all_strategies_pct.py")
strat = importlib.util.module_from_spec(spec)
spec.loader.exec_module(strat)
print("✓ Loaded strategies")

## Load Data

In [ ]:
COMMODITY, MODEL_VERSION = "coffee", "synthetic_acc90"
DATA_PATHS = get_data_paths(COMMODITY, MODEL_VERSION)
COMMODITY_CONFIG = COMMODITY_CONFIGS[COMMODITY]
COMMODITY_CONFIG["storage_cost_pct_per_day"] = 0.005
COMMODITY_CONFIG["transaction_cost_pct"] = 0.01
prices = spark.table(get_data_paths(COMMODITY)["prices_prepared"]).toPandas()
prices["date"] = pd.to_datetime(prices["date"])
with open(DATA_PATHS["prediction_matrices"], "rb") as f: prediction_matrices = pickle.load(f)
prediction_matrices = {pd.to_datetime(k): v for k, v in prediction_matrices.items()}
print(f"✓ Data loaded")

## Backtest Engine

In [ ]:
class Engine:
    def __init__(self, prices_df, pred_matrices, config):
        self.prices = prices_df
        self.pred = pred_matrices
        self.config = config
    def run_backtest(self, strategy, inv=50.0):
        inventory, trades, rev, trans, stor = inv, [], 0, 0, 0
        strategy.reset()
        strategy.set_harvest_start(0)
        for day in range(len(self.prices)):
            date = self.prices.iloc[day]["date"]
            price = self.prices.iloc[day]["price"]
            hist = self.prices.iloc[:day+1].copy()
            pred = self.pred.get(date)
            dec = strategy.decide(day=day, inventory=inventory, current_price=price, price_history=hist, predictions=pred)
            if dec["action"] == "SELL" and dec["amount"] > 0:
                amt = min(dec["amount"], inventory)
                r = amt * price * 20
                t = r * self.config["transaction_cost_pct"] / 100
                rev += r
                trans += t
                inventory -= amt
            if inventory > 0:
                stor += inventory * self.prices.iloc[:day+1]["price"].mean() * 20 * self.config["storage_cost_pct_per_day"] / 100
        return {"net_earnings": rev - trans - stor, "total_revenue": rev, "num_trades": len(trades), "storage_costs": stor, "final_inventory": inventory}
engine = Engine(prices, prediction_matrices, COMMODITY_CONFIG)
print("✓ Engine ready")

## Search Spaces

In [ ]:
def get_params(t, s):
    if s == "immediate_sale": return {"min_batch_size": t.suggest_float("min_batch_size", 3.0, 10.0), "sale_frequency_days": t.suggest_int("sale_frequency_days", 5, 14)}
    elif s == "equal_batch": return {"batch_size": t.suggest_float("batch_size", 0.15, 0.30), "frequency_days": t.suggest_int("frequency_days", 20, 35)}
    elif s == "price_threshold": return {"threshold_pct": t.suggest_float("threshold_pct", 0.02, 0.07), "batch_baseline": t.suggest_float("batch_baseline", 0.20, 0.35), "batch_overbought_strong": t.suggest_float("batch_overbought_strong", 0.30, 0.40), "batch_overbought": t.suggest_float("batch_overbought", 0.25, 0.35), "batch_strong_trend": t.suggest_float("batch_strong_trend", 0.15, 0.25), "rsi_overbought": t.suggest_int("rsi_overbought", 65, 75), "rsi_moderate": t.suggest_int("rsi_moderate", 60, 70), "adx_strong": t.suggest_int("adx_strong", 20, 30), "cooldown_days": t.suggest_int("cooldown_days", 5, 10), "max_days_without_sale": t.suggest_int("max_days_without_sale", 45, 75)}
    elif s == "moving_average": return {"ma_period": t.suggest_int("ma_period", 20, 35), "batch_baseline": t.suggest_float("batch_baseline", 0.20, 0.30), "batch_strong_momentum": t.suggest_float("batch_strong_momentum", 0.15, 0.25), "batch_overbought": t.suggest_float("batch_overbought", 0.25, 0.35), "batch_overbought_strong": t.suggest_float("batch_overbought_strong", 0.30, 0.40), "rsi_overbought": t.suggest_int("rsi_overbought", 65, 75), "rsi_min": t.suggest_int("rsi_min", 40, 50), "adx_strong": t.suggest_int("adx_strong", 20, 30), "adx_weak": t.suggest_int("adx_weak", 15, 25), "cooldown_days": t.suggest_int("cooldown_days", 5, 10), "max_days_without_sale": t.suggest_int("max_days_without_sale", 45, 75)}
    else: raise ValueError(s)
print("✓ Search spaces")

## Optimize Function - PURE IN-MEMORY

In [ ]:
def opt(cls, name, n=200):
    print(f"\n{'='*80}\n{name}: {n} trials\n{'='*80}")
    study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=42))
    def obj(trial):
        p = get_params(trial, name)
        if name not in ["immediate_sale", "equal_batch", "price_threshold", "moving_average"]:
            p["storage_cost_pct_per_day"] = COMMODITY_CONFIG["storage_cost_pct_per_day"]
            p["transaction_cost_pct"] = COMMODITY_CONFIG["transaction_cost_pct"]
        try:
            return cls(**p).reset() or engine.run_backtest(cls(**p))["net_earnings"]
        except: return -1e9
    study.optimize(obj, n_trials=n, show_progress_bar=True)
    print(f"Best: ${study.best_value:,.2f}")
    return study.best_params, study
print("✓ Optimize ready")

## Run All Strategies

In [ ]:
results = {}
for name, cls in [("immediate_sale", strat.ImmediateSaleStrategy), ("equal_batch", strat.EqualBatchStrategy), ("price_threshold", strat.PriceThresholdStrategy), ("moving_average", strat.MovingAverageStrategy)]:
    p, s = opt(cls, name, 200)
    results[name] = (p, s.best_value)
print("\n✓ Done!")

## Summary

In [ ]:
for n, (p, v) in results.items():
    print(f"{n}: ${v:,.2f}")